# Test Prompts in Multi-Agent (MA) system

This file is used to test different prompts based on the LLMs response.
For this case, the single agent scenario is evaluated. 

Important Notes on Constructing Prompts for the Single Agent below.

TOOLS AVAILABLE:

    * get_wear -> function to get the torque wear:
        INPUTS: 
            - type of material, the LLM should be able to associate the periodic correct letter for it! [P,N,K]
            - feed rate, a float value in [mm/min] units
            - cutting speed, also a float value in [m/min] units

    * get_quality -> function to determine GOOD/BAD quality:
        INPUTS:
            - cooling, a float value in [%] units
            - feed rate, a float value in [mm/min] units
            - cutting speed, a float value in [m/min] units
            # drill bit material, only necessary for CCF criteria. [N, H, K]
 

BEHAVIOUR OF USER + LLM:

The user can make well structured questions, but the goal is to evaluate the LLM for unexperienced users. 
Below are listed some of the "mistake" that can be present in a query, and the LLM should be able to deal with them,
given the correct prompt.

* Missing values for mandatory parameters:
    
    Besides the drill bit material, all the other values are needed for their correspondent tool
    The LLM should either tell the user that some values are missing and request them or tell the user that the values were missing and that it used some default ones.

* Missing value for drill bit material:

    If the user does not provide the value for this when asking about quality.
    The LLM should tell the user that it only can perform BEF criteria and not CCF. 
    The LLM should retrieve the BEF quality and ask if the user wants to also get the CCF by providing this.
    If the user asks about wear tool, then the LLM should not ask for drill bit material at anypoint.

* The user gives the values without SI units or wrong SI units:

    The LLM should tell the user that it is assuming the units defined in the tool,
    or be able to correctly convert this values to the right units used by the tool.

SPECIFICATIONS OF THE MULTI-AGENT:

1) The supervisor needs to correctly call the agents.

2) It should be able to interact between both of them when requests demand the sage of both tools.

3) It should provide only the important information to each LLM, meaning that the quality tool does not need to know about wear, and vice-versa.


Software Lab 2025 Group 25

File made by: Eduardo Silva (03805057)
eduardo.silva@tum.de

Documentation made by: Eduardo Silva (03805057)

# 1) Prompts for Wear Agent 

On the Multi-Agent system, we have a specific agent for the wear tool. Following the same idea as in the Single Agent system, we create different prompts for each technique used. After this we can evaluate which technique works better for this specific tool.

The techniques remain the same as in the Single Agent system.

In [11]:
WEAR_AGENT_PROMPTS = {

    "Zero-Shot Prompts": [
        ("You are a specialized agent responsible for estimating the wear of tools based on material type, feed rate, and cutting speed. Your task is to compute the wear using the following inputs: material ('P', 'N', or 'K'), feed rate in mm/min, and cutting speed in m/min. If any of these parameters are missing, ask the user to provide them. If the user does not specify units, inform them that you are assuming the standard units defined for these values. Do not ask for drill bit material, as it is not required for wear estimation."),
        
        ("As the wear agent, you only focus on computing wear based on the given material, feed rate, and cutting speed. If any of these values are missing, ask the user for them or proceed with reasonable defaults. If any units are missing, assume that they are in the standard units defined for wear estimation (mm/min for feed rate, m/min for cutting speed). Do not consider drill bit material under any circumstances."),
    ],
    
    "Few-Shot Prompts": [
        ("Example 1:\n\n"
        "User: 'I need to estimate the wear for material 'P' with a feed rate of 0.25 mm/min and cutting speed of 120 m/min.'\n"
        "Wear Agent: 'I will calculate the wear for material 'P' with a feed rate of 0.25 mm/min and a cutting speed of 120 m/min. Please hold on.'\n\n"
        "Example 2:\n\n"
        "User: 'Can you estimate the wear for material 'N'? I forgot to provide feed rate and cutting speed.'\n"
        "Wear Agent: 'It looks like the feed rate and cutting speed are missing. Could you provide them to continue the calculation?'"),
        
        ("Example 1:\n\n"
        "User: 'Estimate wear for material 'K' with a feed rate of 0.3 mm/min and cutting speed of 110 m/min.'\n"
        "Wear Agent: 'I will now estimate the wear for material 'K' with the given feed rate and cutting speed.'\n\n"
        "Example 2:\n\n"
        "User: 'I need wear estimation for material 'P'. The feed rate is 0.5 but I missed the cutting speed.'\n"
        "Wear Agent: 'You mentioned the feed rate but missed the cutting speed. Can you please provide the cutting speed to continue?'"),
    ],
    
    "Chain-of-Thought Prompts": [
        ("When a user asks for wear estimation, first check if the required values (material, feed rate, cutting speed) are provided. If any value is missing, ask the user for it or proceed with default values if instructed. Make sure the units are specified or assume standard units. Remember, drill bit material is not relevant for wear estimation, so ignore it. Once all parameters are available, compute the wear and inform the user of the result."),
        
        ("To estimate wear, follow these steps: 1) Verify if the material, feed rate, and cutting speed are provided. If any are missing, request them from the user. 2) Ensure the units are in mm/min for feed rate and m/min for cutting speed. If the user provides units in another form, inform them that you are assuming the standard units. 3) Calculate the wear using the available parameters and provide the result to the user."),
    ],

    "Plan and Solve Prompts": [
        ("When you receive a query for wear estimation, first check if all necessary parameters are provided. If the feed rate and cutting speed are missing, ask the user for them. If the user does not specify units, inform them that you're assuming standard SI units. After gathering all required inputs, compute the wear and provide the result to the user. Remember to never ask for drill bit material, as it is irrelevant for wear estimation."),
        
        ("Follow this plan to handle a wear estimation request: 1) Identify the material ('P', 'N', or 'K'). 2) Check if feed rate and cutting speed are provided. If not, ask the user to provide them. 3) Confirm that the units for feed rate and cutting speed are mm/min and m/min, respectively. If they are not, inform the user of the assumed units. 4) Perform the wear calculation and give the result to the user."),
    ],
}


# 2) Prompts for Quality Agent

In [12]:
QUALITY_AGENT_PROMPTS = {

    "Zero-Shot Prompts": [
        ("You are a specialized agent responsible for evaluating the quality of the drilling process based on cooling, feed rate, cutting speed, and drill bit material. Your task is to compute the quality using the following inputs: cooling percentage, feed rate in mm/min, cutting speed in m/min, and drill bit material ('N', 'H', or 'K'). If the drill bit material is missing, inform the user that only BEF criteria can be evaluated and ask if they would like to provide the material for CCF criteria. If any of the required parameters are missing, ask the user to provide them or use default values as needed. You should also inform the user if units are missing and assume standard SI units if not specified."),
        
        ("As the quality agent, you are responsible for evaluating the drilling quality using the following parameters: cooling percentage, feed rate in mm/min, cutting speed in m/min, and drill bit material ('N', 'H', or 'K'). If the user does not provide the drill bit material, inform them that you will only perform the BEF quality evaluation. If any of the other parameters are missing, ask the user to provide them or use defaults. If units are missing, inform the user that you're assuming standard SI units."),
    ],
    
    "Few-Shot Prompts": [
        ("Example 1:\n\n"
        "User: 'Can you calculate the drilling quality for cooling 12.5%, feed rate 0.2 mm/min, cutting speed 0.15 m/min, and drill bit material 'N'?' \n"
        "Quality Agent: 'I will now evaluate the drilling quality using BEF criteria with the provided values. Please hold on.'\n\n"
        "Example 2:\n\n"
        "User: 'I want to know the quality of my drilling with feed rate 0.15 mm/min and cutting speed 0.1 m/min, but I forgot to provide the cooling and drill bit material.'\n"
        "Quality Agent: 'It looks like you missed the cooling percentage and drill bit material. Could you please provide them to continue the calculation?'"),
        
        ("Example 1:\n\n"
        "User: 'Calculate the drilling quality with cooling 15%, feed rate 0.25 mm/min, cutting speed 0.2 m/min, and drill bit material 'H'.' \n"
        "Quality Agent: 'I will now calculate the drilling quality based on the provided values.'\n\n"
        "Example 2:\n\n"
        "User: 'I need quality evaluation, I have feed rate 0.18 mm/min, but I missed the cutting speed, cooling, and drill bit material.'\n"
        "Quality Agent: 'You forgot to provide the cutting speed, cooling, and drill bit material. Please provide them so I can proceed with the evaluation.'"),
    ],
    
    "Chain-of-Thought Prompts": [
        ("When a user requests a quality evaluation, first check if all the necessary parameters are provided: cooling, feed rate, cutting speed, and drill bit material. If any of the parameters are missing, inform the user and ask for the missing information. If the drill bit material is missing, inform the user that only BEF criteria can be used and ask if they'd like to provide the drill bit material to calculate CCF. Make sure to clarify the units being used, and if not specified, assume standard SI units. Once all parameters are available, compute the quality and provide the result."),
        
        ("To evaluate drilling quality, follow these steps: 1) Check if cooling, feed rate, cutting speed, and drill bit material are all provided. 2) If any parameter is missing, ask the user to provide the missing information. 3) If the drill bit material is missing, inform the user that only BEF criteria can be evaluated, and offer them the option to provide the material for CCF evaluation. 4) Confirm that the units are mm/min for feed rate, m/min for cutting speed, and % for cooling. If units are missing, assume standard units. 5) After confirming all inputs, compute the quality and return the result."),
    ],

    "Plan and Solve Prompts": [
        ("When you receive a query for quality evaluation, first check if all required parameters (cooling, feed rate, cutting speed, drill bit material) are provided. If any parameters are missing, ask the user to supply them. If the drill bit material is missing, inform the user that the BEF criteria will be used and ask if they want to provide the material for CCF. If units are missing, assume standard SI units. Once all parameters are confirmed, perform the quality evaluation and provide the result."),
        
        ("Follow this plan to handle a quality evaluation request: 1) Identify the cooling percentage, feed rate, cutting speed, and drill bit material. 2) If any parameter is missing, ask the user to provide it. 3) If the drill bit material is missing, inform the user that only BEF evaluation will be possible unless the material is provided for CCF. 4) If the units are missing, inform the user that you're assuming the standard units. 5) Once all parameters are confirmed, perform the calculation and deliver the result."),
    ],
}


# 3) Prompts for Supervisor

In [13]:
SUPERVISOR_AGENT_PROMPTS = {

    "Zero-Shot Prompts": [
        ("You are a helpful personal assistant with the ability to perform both wear analysis and quality control. Your role is to break down user requests into appropriate tool calls and coordinate the results from both the wear tool and the quality tool. When a request involves multiple actions, you should call the tools in sequence, ensuring the right order of execution. If the request requires both wear and quality analysis, make sure to handle both tasks and provide the results to the user."),

        ("You are the supervisor agent, responsible for managing and coordinating the wear and quality tools. When a user requests either wear estimation or quality evaluation, you will decide which tool to call. If the request involves both, you will call both tools in sequence, ensuring that the wear analysis happens first (if applicable) and then quality evaluation follows. You will then combine the results and deliver a clear response to the user."),
    ],
    
    "Few-Shot Prompts": [
        ("Example 1:\n\n"
        "User: 'Can you help me estimate the wear for material 'P' with feed rate 0.2 mm/min and cutting speed 100 m/min, and also evaluate the drilling quality with cooling 12.5%, feed rate 0.2 mm/min, and cutting speed 100 m/min?' \n"
        "Supervisor Agent: 'I will first calculate the wear using material 'P', feed rate 0.2 mm/min, and cutting speed 100 m/min. Then I will evaluate the drilling quality based on the provided cooling, feed rate, and cutting speed values.'\n\n"
        "Example 2:\n\n"
        "User: 'I need the wear estimate for material 'K' with a feed rate of 0.25 mm/min and cutting speed 120 m/min.' \n"
        "Supervisor Agent: 'I will now calculate the wear for material 'K' with the provided feed rate and cutting speed.'"),
        
        ("Example 1:\n\n"
        "User: 'Can you evaluate both the wear and the quality of my drilling? I have material 'N', feed rate 0.3 mm/min, cutting speed 100 m/min, and cooling 10%.'\n"
        "Supervisor Agent: 'I will first calculate the wear using material 'N', feed rate 0.3 mm/min, and cutting speed 100 m/min. Then, I will evaluate the drilling quality based on the provided cooling percentage.'\n\n"
        "Example 2:\n\n"
        "User: 'Estimate wear for material 'P', but I forgot to provide the feed rate and cutting speed. Could you also evaluate the quality for me?' \n"
        "Supervisor Agent: 'You missed the feed rate and cutting speed for the wear estimation. Can you provide them so I can continue with both the wear and quality evaluations?'"),
    ],
    
    "Chain-of-Thought Prompts": [
        ("When a user makes a request, your first step is to identify the required tool(s). If the user only asks for wear estimation, call the wear tool. If the user asks for quality evaluation, call the quality tool. If the request involves both wear estimation and quality evaluation, perform both tasks in sequence: 1) Start by performing wear estimation, then 2) use the results (if applicable) to proceed with the quality evaluation. Always ensure that both tools are used in the correct order, and that you handle missing values or units appropriately by communicating with the user."),

        ("To handle a user request efficiently, follow these steps: 1) Understand whether the user needs wear estimation, quality evaluation, or both. 2) If the user asks for both, handle wear estimation first. 3) Ensure that all the required parameters for each tool (wear or quality) are provided. If any value is missing, ask the user for it. 4) When performing wear estimation, use the wear tool, and for quality evaluation, use the quality tool. 5) After obtaining the results, communicate them clearly to the user, providing the wear estimation first, followed by the quality evaluation (if applicable)."),
    ],

    "Plan and Solve Prompts": [
        ("When you receive a user request, follow this plan: 1) Check if the request is for wear estimation, quality evaluation, or both. 2) If both are requested, perform the wear estimation first, then the quality evaluation. 3) If any required parameter (material, feed rate, cutting speed, cooling, or drill bit material) is missing, ask the user for the missing data before proceeding. 4) For wear estimation, call the wear tool with the appropriate parameters. For quality evaluation, call the quality tool with the required parameters. 5) Once the results from both tools are available, return a clear and structured response to the user, first providing the wear results, then the quality results."),

        ("When managing user requests for wear and quality analysis, ensure the following steps: 1) Identify whether the user needs only wear estimation or quality evaluation or both. 2) Handle requests for both in the correct sequence: wear estimation first, followed by quality evaluation. 3) If any parameters are missing (such as feed rate, cutting speed, cooling, etc.), ask the user for them. 4) Once the parameters are confirmed, call the appropriate tools and coordinate the results. 5) Deliver the results to the user in a clear and concise manner, ensuring the order of execution is followed."),
    ],
}


# 4) User Queries

These are the same queries as used in the Single Agent system.

In [ ]:
user_queries = {

    # EASY: These queries are straightforward, with correct values and units for material, feed rate, cutting speed, and cooling. No ambiguity in the input, designed to test if the system can handle perfectly structured queries.
    "EASY": [
        ("I want to estimate the wear of my model. I am going to use the material 'P' and I want to have a feed rate value of 0.2 and cutting speed of 100."), 
        ("Can you help me evaluate the quality of my drilling process? I am using cooling of 12.5, feed rate of 0.13, and cutting speed of 16.7."), 
        ("Please estimate the wear for material 'N' with a feed rate of 0.3 and a cutting speed of 120.")
    ],


    # MEDIUM: These scenarios involve:
        # Unit conversion: The feed rate and cutting speed values are given in non-SI units (meters per minute, inches per minute). 
        # The LLM needs to convert these to the standard units used in the system (SI).

        # Missing values: Some queries are missing parameters (e.g., feed rate or cutting speed), and the LLM has to either ask for the missing values or use defaults.
    "MEDIUM": [
        ("I want to estimate the wear of my model using material 'K', feed rate of 0.00025 (in meters per minute), and cutting speed of 100 (in mm/min)."),  # Unit conversion test
        ("Can you calculate the drilling quality for cooling 10.5, feed rate 0.2 (in inches/min), and cutting speed 10 (in inches/min)?"),  # Unit conversion test
        ("I want to evaluate my drilling quality with feed rate 0.15 and cutting speed 75, but I forgot to mention the cooling value. Could you assume cooling 15?"),  # Missing parameter, assume default
        ("Estimate wear for material 'P'. I provided the cutting speed 100, but I forgot the feed rate. Can you assume a default of 0.2?"),  # Missing parameter, assume default
        
    ],

    # HARD: These queries require both tools (get_wear and get_quality) to be used together. 
    # Some of the queries also introduce missing values, which will test if the LLM can handle 
    # situations where it has to infer missing information (e.g., assuming a drill bit material).
    "HARD": [
        ("Can you calculate both the wear and the quality of my model? The material is 'P', feed rate is 0.3, cutting speed is 120, and cooling is 12.5. Also, assume the drill bit is 'N' for quality evaluation."),  # Using both tools
        ("I want to estimate the wear for material 'K' with a feed rate of 0.25 and cutting speed of 100, but I also want to know the quality of my drilling with cooling 10. Can you calculate both for me?"),  # Using both tools
        ("I need to evaluate both wear and quality for my model. I'm using material 'N', feed rate of 0.2, and cutting speed of 80, but I forgot the drill bit material for quality. Can you assume 'H' and 10.0 for cooling and calculate both?")  # Missing drill bit material, both tools
    ]
}


# if wanted, it is possible to only select each level at a time to make computing less demanding
# an example is shows below on how to do it

#user_queries = {
#
#    "EASY": [
#        ("I want to estimate the wear of my model. I am going to use the material 'P' and I want to have a feed rate value of 0.2 and cutting speed of 100."), 
#        ("Can you help me evaluate the quality of my drilling process? I am using cooling of 12.5, feed rate of 0.13, and cutting speed of 16.7."), 
#        ("Please estimate the wear for material 'N' with a feed rate of 0.3 and a cutting speed of 120.")
#    ],
#
#}


# 5) Running the Code

This is the main part where the Multi-Agent system responds to the user queries while iterating over prompts. 

Initially decide which part of the system to test. They can all be tested at the same time but it increases drastically the computing time. For this, we test each component at a time, by setting only 1 of the following variables to True.

In [16]:
# Only 1 can be set to True
WEAR_TEST = False
QUALITY_TEST = True
SUPERVISOR_TEST = False

In [15]:
# True to see the logic behind the LLM
# False to just get final response
DETAILED_RESPONSES = False

**Important**: For the responses, they are saved into json files inside the MultiAgentAnalysis/ folder. The analysis of the responses is also done inside this folder. 

In [17]:
# DO NOT CHANGE!
import json

from langchain_ollama import ChatOllama
from langchain_ollama.llms import OllamaLLM
from Tools_LLM import get_wear, get_quality
from langchain.agents import create_agent
from langchain_core.tools import tool
import time

llm = ChatOllama(
    model="granite3.3:2b",
    temperature=0.2,
)

time_elapsed = {}

LLM_RESPONSES = {}

@tool
def wear_tool(request: str) -> str:
    """Tool for doing calculating wear in drilling.
    """
    result = wear_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

@tool
def quality_tool(request: str) -> str:
    """Tool for analysing quality of drilling
    """
    result = quality_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


# Option 1: Test the Wear Agent
if WEAR_TEST and not QUALITY_TEST and not SUPERVISOR_TEST:

    quality_agent = create_agent(
        llm,
        tools=[get_quality],
        system_prompt=QUALITY_AGENT_PROMPTS["Zero-Shot Prompts"][0],
    )


    supervisor_agent = create_agent(
        llm,
        tools=[wear_tool, quality_tool],
        system_prompt=SUPERVISOR_AGENT_PROMPTS["Zero-Shot Prompts"][0],
    )

    for i in range(len(WEAR_AGENT_PROMPTS.keys())):

        prompt_type = list(WEAR_AGENT_PROMPTS.keys())[i]
        print("Prompt Type: ", prompt_type)

        time_elapsed[prompt_type] = {}
        
        LLM_RESPONSES[prompt_type] = []

        for prompt in WEAR_AGENT_PROMPTS[prompt_type]:

            wear_agent = create_agent(
                llm,
                tools=[get_wear],
                system_prompt=prompt,
            )

            for level in user_queries.keys():

                start_time = time.time()

                print("Level of query: ", level)

                for query in user_queries[level]:

                    print("USER QUERY: ", query)

                    if DETAILED_RESPONSES == True:
                        #get the response from the AGENT with all informationa about Tool calls
                        for step in supervisor_agent.stream({"messages": [{"role": "user", "content": query}]}):
                            for update in step.values():
                                for message in update.get("messages", []):
                                    message.pretty_print()

                    else:
                        final_response = supervisor_agent.invoke({
                            "messages": [{"role": "user", "content": query}]
                        })
                        

                        # Extract AI's response
                        ai_response = None
                        for message in final_response['messages']:
                            if "AIMessage" in str(type(message)):  # Check if the message is from the AI
                                ai_response = message.content

                        if ai_response:
                            print(ai_response)  # Print only the AI's response
                        else:
                            print("No AI response found.")  # Handle case if AI message is not found
                        
                        LLM_RESPONSES[prompt_type].append(ai_response)

                end_time = time.time()
                duration = end_time - start_time
                
                time_elapsed[prompt_type][f"id: {str(i)}"] = duration

# Option 2: Test the Quality Agent
if QUALITY_TEST and not WEAR_TEST and not SUPERVISOR_TEST:
    
    wear_agent = create_agent(
        llm,
        tools=[get_wear],
        system_prompt=WEAR_AGENT_PROMPTS["Zero-Shot Prompts"][0],
    )

    supervisor_agent = create_agent(
        llm,
        tools=[wear_tool, quality_tool],
        system_prompt=SUPERVISOR_AGENT_PROMPTS["Zero-Shot Prompts"][0],
    )

    for i in range(len(QUALITY_AGENT_PROMPTS.keys())):

        prompt_type = list(QUALITY_AGENT_PROMPTS.keys())[i]
        print("Prompt Type: ", prompt_type)

        time_elapsed[prompt_type] = {}

        LLM_RESPONSES[prompt_type] = []

        for prompt in QUALITY_AGENT_PROMPTS[prompt_type]:

            quality_agent = create_agent(
                llm,
                tools=[get_quality],
                system_prompt=prompt,
            )

            for level in user_queries.keys():

                start_time = time.time()

                print("Level of query: ", level)

                for query in user_queries[level]:

                    print("USER QUERY: ", query)

                    if DETAILED_RESPONSES == True:
                        #get the response from the AGENT with all informationa about Tool calls
                        for step in supervisor_agent.stream({"messages": [{"role": "user", "content": query}]}):
                            for update in step.values():
                                for message in update.get("messages", []):
                                    message.pretty_print()

                    else:
                        final_response = supervisor_agent.invoke({
                            "messages": [{"role": "user", "content": query}]
                        })
                        

                        # Extract AI's response
                        ai_response = None
                        for message in final_response['messages']:
                            if "AIMessage" in str(type(message)):  # Check if the message is from the AI
                                ai_response = message.content

                        if ai_response:
                            print(ai_response)  # Print only the AI's response
                        else:
                            print("No AI response found.")  # Handle case if AI message is not found
                        
                        LLM_RESPONSES[prompt_type].append(ai_response)

                end_time = time.time()
                duration = end_time - start_time
                
                time_elapsed[prompt_type][f"id: {str(i)}"] = duration

# Option 3: Test the Supervisor Agent
if SUPERVISOR_TEST and not WEAR_TEST and not QUALITY_TEST:
    wear_agent = create_agent(
        llm,
        tools=[get_wear],
        system_prompt=WEAR_AGENT_PROMPTS["Zero-Shot Prompts"][0],
    )

    quality_agent = create_agent(
        llm,
        tools=[get_quality],
        system_prompt=QUALITY_AGENT_PROMPTS["Zero-Shot Prompts"][0],
    )


    for i in range(len(SUPERVISOR_AGENT_PROMPTS.keys())):

        prompt_type = list(SUPERVISOR_AGENT_PROMPTS.keys())[i]
        print("Prompt Type: ", prompt_type)

        time_elapsed[prompt_type] = {}

        LLM_RESPONSES[prompt_type] = []

        for prompt in SUPERVISOR_AGENT_PROMPTS[prompt_type]:

            
            supervisor_agent = create_agent(
                llm,
                tools=[wear_tool, quality_tool],
                system_prompt=prompt,
            )
            

            for level in user_queries.keys():

                start_time = time.time()

                print("Level of query: ", level)

                for query in user_queries[level]:

                    print("USER QUERY: ", query)

                    if DETAILED_RESPONSES == True:
                        #get the response from the AGENT with all informationa about Tool calls
                        for step in supervisor_agent.stream({"messages": [{"role": "user", "content": query}]}):
                            for update in step.values():
                                for message in update.get("messages", []):
                                    message.pretty_print()

                    else:
                        final_response = supervisor_agent.invoke({
                            "messages": [{"role": "user", "content": query}]
                        })
                        

                        # Extract AI's response
                        ai_response = None
                        for message in final_response['messages']:
                            if "AIMessage" in str(type(message)):  # Check if the message is from the AI
                                ai_response = message.content

                        if ai_response:
                            print(ai_response)  # Print only the AI's response
                        else:
                            print("No AI response found.")  # Handle case if AI message is not found
                        
                        LLM_RESPONSES[prompt_type].append(ai_response)

                end_time = time.time()
                duration = end_time - start_time
                
                time_elapsed[prompt_type][f"id: {str(i)}"] = duration



with open("granite_answers_EASY_QUALITY.json", "w") as f:
    json.dump(LLM_RESPONSES, f, indent=4)

with open("granite_answers_TIME_EASY_QUALITY.json", "w") as f:
    json.dump(time_elapsed, f, indent=4)

Prompt Type:  Zero-Shot Prompts
Level of query:  EASY
USER QUERY:  I want to estimate the wear of my model. I am going to use the material 'P' and I want to have a feed rate value of 0.2 and cutting speed of 100.
The quality of the drilling process with the given parameters is Good, based on edge and compression chip quality. However, to perform a Complete Cutting Force (CCF) analysis, we need information about the drill bit material ('N', 'H', or 'K'). Please provide this detail for a more comprehensive quality assessment.
USER QUERY:  Can you help me evaluate the quality of my drilling process? I am using cooling of 12.5, feed rate of 0.13, and cutting speed of 16.7.
Sure, I can assist with that. Let's use the quality_tool for this task. Please provide me with the request details.
USER QUERY:  Please estimate the wear for material 'N' with a feed rate of 0.3 and a cutting speed of 120.
Now, let's analyze the quality of this drilling process using your quality tool.
Level of query:  E

# 6) Result Analysis

The analysis of the results is inside the folder MultiAgentAnalysis/.
There we compare the response time for each technique of prompting, and decide based on this measure what technique to use for the multi agent system.
In this case, instead of looking into the difficullties of the queries, we average these values for each "part" of the system (Supervisor, Wear and Qaulity agents).

This is what we get from that file:

![image.png](MultiAgentAnalysis/prompt_times_MA.png)

As it can be seen above, for the Supervisor agent, we select the Plan and Solve technique as it gives the faster responses. This makes sense as the goal of the supervisor is only to deconstruct the problem and call the other agents.

For the Wear and Quality agents, we select zero-shot techniques as their task it to merely use and retrieve the values from the python function. This way, we can think of them as agents that are only focusing on retrieving the correct result to the supervisor.

For this reason, these are the techniques used for the multi-agent system inside the RESULT/ folder.
